In [ ]:
from src.tools import MMseqs, NCBIDatasets, UniRef
import pandas as pd
import numpy as np
from src.files import XMLFile, InterProScanFile, FASTAFile, GBFFFile
from utils import * 
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from src.classifier import Classifier
from src.sampler import Sampler
from src.genome import ReferenceGenome
from src.dataset import Dataset
import re
from src.build import * 

%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ncbi_genome_metadata_df = load_ncbi_genome_metadata()

for level in ['phylum', 'class', 'order', 'genus']:
    print(f'{level.capitalize()} count:', ncbi_genome_metadata_df[f'{level}'].nunique())
    # print(f'Species with no {level}:', (genome_metadata_df[f'{level}_taxid'] == 'none').sum())

# fig, (ax_a, ax_b) = plt.subplots(ncols=2, figsize=(15, 5), width_ratios=[0.3, 0.7])

# counts, bins, _ = ax_a.hist(ncbi_genome_metadata_df.gc_percent, bins=25, width=2, edgecolor='black', color='lightgray')
# ax_a.set_ylabel('density')
# ax_a.set_xlabel('GC percent')

# level = 'phylum'
# # x = np.arange(genome_metadata_df[f'{level}_taxid'].nunique())
# y = ncbi_genome_metadata_df.groupby(level).gc_percent.mean() # .sort_values(ascending=True)
# x = ncbi_genome_metadata_df.groupby(level).apply(len, include_groups=False) # .sort_values(ascending=True)
# y_err = ncbi_genome_metadata_df.groupby(level).apply(lambda df : df.gc_percent.std() / np.sqrt(len(df)), include_groups=False) # .loc[y.index]

# ax_b.errorbar(x, y, yerr=y_err, ls='', lw=0.7, capsize=2, color='black')
# ax_b.scatter(x, y, color='black', s=5)
# ax_b.set_xscale('log')
# ax_b.set_xlabel(f'log({level} size)')
# ax_b.set_ylabel('GC percent')

plt.show()

Phylum count: 56
Class count: 123
Order count: 299
Genus count: 3983


In [3]:
# How should I select the genomes for model training?

# Using the same genomes for validation as in the Prodigal paper
# Halobacterium salinarum
# Natronomonas pharaonis
# Aeropyrum pernix
model_organism_genome_ids = ['GCF_000005845.2'] # E. coli K-12
model_organism_genome_ids += ['GCF_000009045.1'] # B. subtilis
model_organism_genome_ids += ['GCF_000006765.1'] # P. aeruginosa

# Start by selecting a few phyla in a reasonable size range with varying GC contents.
def sample(genome_metadata_df:pd.DataFrame, min_phylum_size:int=90, max_phylum_size:int=200):

    phylum_sizes = genome_metadata_df.groupby('phylum').apply(len, include_groups=False)
    phylum_sizes = phylum_sizes[(phylum_sizes > min_phylum_size) & (phylum_sizes < max_phylum_size)]

    genome_metadata_df = genome_metadata_df[genome_metadata_df.phylum.isin(phylum_sizes.index)]
    
    phylum_gc_percent = genome_metadata_df.groupby('phylum').gc_percent.mean() 
    phylum_gc_percent = phylum_gc_percent.sort_values(ascending=True)
    
    phyla = dict()
    phyla['low_gc_content'] = phylum_gc_percent.index[0]
    phyla['med_gc_content'] = phylum_gc_percent.index[len(phylum_gc_percent) // 2]
    phyla['high_gc_content'] = phylum_gc_percent.index[-1]

    print('Low-GC content phylum:', phyla['low_gc_content'])
    print('Medium-GC content phylum:', phyla['med_gc_content'])
    print('High-GC content phylum:', phyla['high_gc_content'])

    genome_ids = {category:genome_metadata_df.index[genome_metadata_df.phylum == phylum] for category, phylum in phyla.items()}
    return genome_ids

genome_ids = sample(ncbi_genome_metadata_df)

# ncbi = NCBIDatasets(genome_dir='../data/genomes', gbff_dir='../data/proteins/ncbi')
# ncbi.run(genome_ids=genome_ids['low_gc_content'], metadata_only=False)

Low-GC content phylum: Campylobacterota
Medium-GC content phylum: Planctomycetota
High-GC content phylum: Deinococcota


In [4]:
genome_metadata_df = list()
for category, genome_ids_ in genome_ids.items():
    df = ncbi_genome_metadata_df.loc[genome_ids_, :].copy()
    df['gc_percent_category'] = category
    # df = df[genome_metadata_df.annotation_pipeline.str.contains('PGAP')] # Only include genomes annotated with PGAP, because the other files are not consistent. 
    genome_metadata_df.append(df)
genome_metadata_df = pd.concat(genome_metadata_df)
# genome_metadata_df.to_csv('../data/genome_metadata.csv')

In [5]:
# from src.tools import download_homologs
# for path in glob.glob('../data/proteins/ncbi/*'):
#     download_homologs(path, pseudo_only=True, overwrite=False)

In [ ]:
for genome_id in genome_ids['med_gc_content'][::-1]:
    ! ref --input-path ../data/proteins/prodigal/{genome_id}_protein.faa --output-dir ../data/ref --reference-dir ../data/proteins/ncbi/ --prodigal-output --summarize --load-homologs --homologs-dir ../data/proteins/homologs/

In [8]:
ref_df = load_ref(genome_ids=[genome_id for genome_id in genome_metadata_df.index if (genome_id != 'GCF_000009085.1')])

In [ ]:

def check_spurious(ref_df:pd.DataFrame, max_score:float=0.5, remove_invalid:bool=True):
    scores = get_alignment_scores(ref_df, seq_a_col='top_hit_seq', seq_b_col='query_seq')
    # ref_df['alignment_score'] = scores

    valid = ref_df.top_hit_feature.isin(GBFFFile.rna_features + GBFFFile.other_features)
    valid = valid | ((scores < max_score) & ~np.isnan(scores)) 
    ref_df['valid'] = valid

    print(f'check_spurious: Validated {ref_df.valid.sum()} out of {len(ref_df)} spurious sequences.')

    if remove_invalid:
        ref_df = ref_df[ref_df.valid].copy()
        ref_df = ref_df.drop(columns=['valid'])
    return ref_df
    

def check_real(ref_df:pd.DataFrame, min_score:float=0.9, remove_invalid:bool=True):
    scores = get_alignment_scores(ref_df, seq_a_col='top_hit_seq', seq_b_col='query_seq')
    # ref_df['alignment_score'] = scores
    ref_df['valid'] = (scores > min_score) & ~np.isnan(scores)

    print(f'check_real: Validated {ref_df.valid.sum()} out of {len(ref_df)} real sequences.')
    
    if remove_invalid:
        ref_df = ref_df[ref_df.valid].copy()
        ref_df = ref_df.drop(columns=['valid'])
    return ref_df

# It seems that only including things with definitive labels seems to have just increased the number of false negatives. 
# Possibly the benefits of increasing label confidence were outweighed by the shrinkage of the minority class?
    

In [25]:
def build_datasets(genome_metadata_df:pd.DataFrame, phylum:str='Campylobacterota', max_length:int=2000):
    # First remove things which do not have definitive labels. 
    genome_ids = genome_metadata_df[genome_metadata_df.phylum == phylum].index
    genome_ids = [genome_id for genome_id in genome_metadata_df.index if (genome_id != 'GCF_000009085.1')] # Remove the non-PGAP genome. 

    print(f'build_datasets: Loading data from {len(genome_ids)} genomes belonging to the phylum {phylum}.')
    df = load_ref(genome_ids=genome_ids)
    print(f'build_datasets: Loaded {len(ref_df)} sequences.')

    df = pd.concat([check_real(df[df.real].copy()), check_spurious(df[df.spurious].copy())])
    print(f'build_datasets: Kept {df.spurious.sum()} spurious sequences and {df.real.sum()} real sequences.')
    df['label'] = df.real.astype(int)

    lengths = get_lengths(df, top_hit=False)
    print(f'Removing {(lengths >= max_length).sum()} sequences exceeding the maximum length of {max_length}')
    df = df[lengths < max_length]

    df = df.rename(columns={'query_seq':'seq'}) # Need to do this for file writing, etc. to work correctly, 
    df = df.rename(columns={'top_hit_genome_id':'genome_id'}) # Need to do this for file writing, etc. to work correctly, 

    mmseqs = MMseqs()
    df = mmseqs.cluster(df, job_name=f'{phylum.lower()}_95', sequence_identity=0.95, reps_only=True, overwrite=True)
    mmseqs.cleanup()

    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idxs, test_idxs = list(gss.split(df, groups=df.genome_id))[0]
    train_df, test_df = df.iloc[train_idxs], df.iloc[test_idxs]
    print(f'build_datasets: Training dataset contains {len(train_df)} sequences, testing dataset contains {len(test_df)} sequences.')
    print(f'build_datasets: {(train_df.label == 0).sum()} negative instances and {(train_df.label == 1).sum()} positive instances in the training dataset.')
    print(f'build_datasets: {(test_df.label == 0).sum()} negative instances and {(test_df.label == 1).sum()} positive instances in the testing dataset.')
    train_df.to_csv(f'../data/{phylum.lower()}_dataset_train.csv')
    test_df.to_csv(f'../data/{phylum.lower()}_dataset_test.csv')
    return train_df, test_df 
    
# train_df, test_df = build_datasets(genome_metadata_df)
phylum ='Campylobacterota'
dtypes = {'top_hit_partial':str, 'query_partial':str, 'top_hit_translation_table':str, 'top_hit_codon_start':str}
train_df = pd.read_csv(f'../data/{phylum.lower()}_dataset_train.csv', index_col=0, dtype=dtypes)
test_df = pd.read_csv(f'../data/{phylum.lower()}_dataset_test.csv', index_col=0, dtype=dtypes)


In [55]:
pred_df = load_predict('../data/predict/campylobacterota_dataset_test_predict.csv', model_name='campylobacterota_esm_650m_gap_v1')

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

class BenchmarkClassifier():

    def __init__(self, feature_col:str='query_length'):

        self.model = LogisticRegression(class_weight='balanced', C=1) # , multi_class='multinomial') 
        self.scaler = StandardScaler()
        self.feature_col = feature_col

    def predict(self, df:pd.DataFrame, include_outputs:bool=True):
        X = df[[self.feature_col]].values
        X = self.scaler.transform(X)
        model_labels = self.model.predict(X)
        model_outputs = self.model.predict_proba(X)

        return model_labels if (not include_outputs) else (model_labels, model_outputs)

    def fit(self, df:pd.DataFrame):
        X, labels = df[[self.feature_col]].values, df['label'].values
        X = self.scaler.fit_transform(X)
        self.model.fit(X, labels)

model = BenchmarkClassifier()
model.fit(train_df)

df = test_df[['label', 'query_length']].copy()
df['model_label'], outputs = model.predict(test_df, include_outputs=True)
df['model_output_0'], df['model_output_1'] = outputs[:, 0].ravel(), outputs[:, 1].ravel()

print('Balanced accuracy:', balanced_accuracy_score(df.label.values, df.model_label.values))
print('Precision (class 0):', precision(df, class_=0))
print('Recall (class 0):', recall(df, class_=0))
print('Number of false negatives:', ((df.label == 1) & (df.model_label == 0)).sum())


confusion_matrix = np.where((df.model_label == 1) & (df.label == 0), 'false positive', '')
confusion_matrix = np.where((df.model_label  == 1) & (df.label == 1), 'true positive', confusion_matrix)
confusion_matrix = np.where((df.model_label == 0) & (df.label == 1), 'false negative', confusion_matrix)
confusion_matrix = np.where((df.model_label  == 0) & (df.label == 0), 'true negative', confusion_matrix)
df['confusion_matrix'] = confusion_matrix

benchmark_fn = df[df.confusion_matrix == 'false negative'].index
fn = pred_df[pred_df.confusion_matrix == 'false negative'].index
np.intersect1d(fn, benchmark_fn)

Balanced accuracy: 0.8243785650516682
Precision (class 0): 0.0003928135789451922
Recall (class 0): 0.12666666666666668
Number of false negatives: 14003


array(['NC_017735.1_1005', 'NC_017735.1_1006', 'NC_017735.1_1007', ...,
       'NZ_WFKI01000156.1_1', 'NZ_WFKI01000159.1_1',
       'NZ_WFKI01000162.1_1'], shape=(2067,), dtype=object)

In [11]:
# There still seems to be something wrong with the training data, perhaps I am still misclassifying the spurious sequences?
# It is possible my "in-frame" check is wrong, it's possible I should only see if either edge is a multiple of 3 base pairs away, 
# as opposed to both.

# It seems as though 684 of the sequences flagged as spurious have an overlap length of 0, so maybe I should be counting those as intergenic?
# Ok, so I think because of how frequent gene overlap is, I should be allowing some (considering genes with less than a certain amount of overlap to be intergenic)
# I should also be very conservative, i.e. allow a comparatively large amount of overlap. 
# https://pmc.ncbi.nlm.nih.gov/articles/PMC525685/ 


In [8]:
# # Curious about the distribution of overlap sizes... 


# def figure(ref_df:pd.DataFrame, path:str=None, min_overlap_fraction:float=0.8):
#     # ref_df = load_ref(genome_ids=genome_ids)
#     ref_df = ref_df[ref_df.spurious | ref_df.real].copy()

#     fig = plt.figure(figsize=(10, 10), layout='tight')
#     grid = GridSpec(nrows=1, ncols=1, figure=fig)
#     ax_a = fig.add_subplot(grid[0, 0])

#     ax_a_df = pd.DataFrame(index=ref_df.index)
#     ax_a_df['length'] = get_lengths(ref_df, top_hit=False, units='nt')
#     ax_a_df['overlap_length'] = ref_df['top_hit_overlap_length']
#     ax_a_df['overlap_fraction'] = ax_a_df.overlap_length / ax_a_df.length # This is relative to the query sequence. 
#     ax_a_df['label'] = np.select([ref_df.real, ref_df.spurious], ['real', 'spurious'], default='none')
#     if min_overlap_fraction is not None:
#         ax_a_df = ax_a_df[ax_a_df.overlap_fraction > min_overlap_fraction].copy()

#     colors = ['darkseagreen', 'indianred']
#     # sns.kdeplot(data=ax_a_df, ax=ax_a, hue='label', hue_order=['real', 'spurious'], palette=colors, common_norm=False, x='overlap_fraction')
#     sns.ecdfplot(data=ax_a_df, ax=ax_a, hue='label', hue_order=['real', 'spurious'], palette=colors, x='overlap_fraction')

#     plt.show()
#     return ax_a_df


# ax_a_df = figure(ref_df, min_overlap_fraction=None)